<a href="https://colab.research.google.com/github/krishgit042023/Semantic-Movie-Recommendation-System-using-LLM-Embeddings/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/tmdb-movie-metadata


In [3]:
# prompt: load dataset

import pandas as pd
df_credits = pd.read_csv("/content/tmdb_5000_credits.csv")
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
df_credits.shape

(4803, 4)

In [5]:
df_movies = pd.read_csv("/content/tmdb_5000_movies.csv")
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
df_movies.shape

(4803, 20)

In [7]:
# prompt: show the list of columns in df_movies

df_movies.columns


Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [8]:
# prompt: remove 'original title' and 'title' column from df_movies

df_movies = df_movies.drop(columns=['original_title', 'title'], errors='ignore')


In [9]:
df_movies.shape

(4803, 18)

In [10]:
# prompt: join df_credits and df_movies  and name the new dataframe as dataset

import pandas as pd
# Assuming 'id' in df_movies and 'movie_id' in df_credits, correct the merge operation
dataset = pd.merge(df_credits, df_movies, left_on='movie_id', right_on='id', how='inner')

# After the merge, you can drop the redundant 'movie_id' column if you wish
dataset = dataset.drop(columns=['movie_id'])

In [11]:
dataset.head()

,title,cast,crew,budget,genres,homepage,id,keywords,original_language,overview,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800
1,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500
2,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466
3,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,Following the death of District Attorney Harve...,...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106
4,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"John Carter is a war-weary, former military ca...",...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124


In [12]:
dataset.shape

(4803, 21)

In [13]:
dataset.columns

Index(['title', 'cast', 'crew', 'budget', 'genres', 'homepage', 'id',
       'keywords', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'vote_average', 'vote_count'],
      dtype='object')

In [14]:
# prompt: check for null values in dataset dataframe

dataset.isnull().sum()


,0
title,0
cast,0
crew,0
budget,0
genres,0
homepage,3091
id,0
keywords,0
original_language,0
overview,3


In [15]:
# prompt: create a new dataframe consisting of columns 'overview' ,'title', 'genres', 'keywords', 'cast', 'crew' and name the dataframe as movie

movie = dataset[['overview', 'title', 'genres', 'keywords', 'cast', 'crew']]


In [16]:
movie.head()

,overview,title,genres,keywords,cast,crew
0,"In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,A cryptic message from Bond’s past sends him o...,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"John Carter is a war-weary, former military ca...",John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [17]:
movie.shape

(4803, 6)

In [18]:
# prompt: check for null values

movie.isnull().sum()


,0
overview,3
title,0
genres,0
keywords,0
cast,0
crew,0


In [19]:
# prompt: show the row in which the overview column contain null values

import pandas as pd
movie = dataset[['overview', 'title', 'genres', 'keywords', 'cast', 'crew']]
# Find rows where 'overview' is null
null_overview_rows = movie[movie['overview'].isnull()]
null_overview_rows


,overview,title,genres,keywords,cast,crew
2656,NaN,Chiamatemi Francesco - Il Papa della gente,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...","[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4140,NaN,"To Be Frank, Sinatra at 100","[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...","[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4431,NaN,Food Chains,"[{""id"": 99, ""name"": ""Documentary""}]",[],[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [20]:
# prompt: drop the 3 rows in which the overview column contains nan values

movie = movie.dropna(subset=['overview'])
movie.isnull().sum()


,0
overview,0
title,0
genres,0
keywords,0
cast,0
crew,0


In [21]:
movie.shape

(4800, 6)

In [22]:
movie.columns

Index(['overview', 'title', 'genres', 'keywords', 'cast', 'crew'], dtype='object')

**Compute Embeddings**

In [23]:
import pandas as pd

# Function to combine the relevant fields into one text string per movie
def combine_movie_info(row):
    # Ensure each column is converted to string if it's not already
    return f"{row['overview']} {row['title']} {row['genres']} {row['keywords']} {row['cast']} {row['crew']}"

# Create a new column 'combined_text'
movie['combined_text'] = movie.apply(combine_movie_info, axis=1)

# Optionally, inspect the combined text for the first few movies
print(movie['combined_text'].head())


0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: combined_text, dtype: object


<ipython-input-23-ce19fd60699c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['combined_text'] = movie.apply(combine_movie_info, axis=1)


In [24]:
from sentence_transformers import SentenceTransformer
import os

# Specify Hugging Face cache directory using environment variable
os.environ['TRANSFORMERS_CACHE'] = '/content/huggingface_cache'  # Using a custom path within Colab

# Load the pre-trained model, forcing a re-download if needed
model = SentenceTransformer('all-MiniLM-L6-v2') # Remove cache_dir and download_progress_bar

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert the combined text into a list
movie_texts = movie['combined_text'].tolist()

# Compute embeddings for each movie's combined text
movie_embeddings = model.encode(movie_texts)

# Verify the result: print the shape of the embedding vector for the first movie
print("Shape of embedding for the first movie:", movie_embeddings[0].shape)


**Build Query-Based Recommendations**

In [ ]:
user_query = input("Enter your movie preference query (e.g., 'I love mind-bending sci-fi'): ")


#Using the same model to convert the user query into an embedding

query_embedding = model.encode(user_query, convert_to_tensor=True)



from sentence_transformers import util

# Compute cosine similarity between the query embedding and all movie embeddings
cosine_scores = util.cos_sim(query_embedding, movie_embeddings)

# Convert the scores to a numpy array for easier handling
cosine_scores = cosine_scores[0].cpu().numpy()



#display the top 5 recommendations

import numpy as np

# Get the indices of the top 5 movies with the highest cosine similarity scores
top_indices = np.argsort(-cosine_scores)[:5]

print("\nTop 5 Recommended Movies:")
for idx in top_indices:
    title = movie.iloc[idx]['title']  # Assuming your DataFrame 'df' has the movie titles
    score = cosine_scores[idx]
    print(f"{title} (Similarity Score: {score:.4f})")


**Build a Similar-Movie Recommendation Feature**

In [ ]:
from sentence_transformers import util
import numpy as np

# Step 1: Accept user input for a movie title
query_title = input("Enter a movie title (e.g., 'Batman'): ").strip().lower()

# Step 2: Find matching movie in the 'movie' DataFrame
# Using a simple case-insensitive substring search in the 'title' column
matched_movies = movie[movie['title'].str.lower().str.contains(query_title)]

if matched_movies.empty:
    print("No movie found with that title.")
else:
    # For simplicity, take the first matched movie
    selected_movie = matched_movies.iloc[0]
    print(f"Found movie: {selected_movie['title']}")

    # Step 3: Retrieve the index and embedding of the selected movie
    movie_index = selected_movie.name  # Ensure the DataFrame index aligns with the embeddings order
    query_movie_embedding = movie_embeddings[movie_index]

    # Step 4: Compute cosine similarity between the selected movie and all movies
    # The function returns a tensor; we convert it to a numpy array for easier manipulation
    cosine_scores = util.cos_sim(query_movie_embedding, movie_embeddings)[0].cpu().numpy()

    # Step 5: Exclude the selected movie by setting its similarity to -infinity
    cosine_scores[movie_index] = -np.inf

    # Get the indices of the top 5 movies with the highest similarity scores
    top_indices = np.argsort(-cosine_scores)[:5]

    # Step 6: Display the top 5 recommended movies
    print("\nTop 5 movies similar to", selected_movie['title'])
    for idx in top_indices:
        similar_title = movie.iloc[idx]['title']
        score = cosine_scores[idx]
        print(f"{similar_title} (Similarity Score: {score:.4f})")


**Deploying the model**

In [ ]:
!pip install gradio

In [ ]:
# prompt: create a user interface using gradio where the user can give input based on genres or movie name names ,, and in output the top 5 recommendations will be given

import numpy as np
import gradio as gr

def recommend_movies(query_type, input_text):
    if query_type == "Movie Name":
        query_title = input_text.strip().lower()
        matched_movies = movie[movie['title'].str.lower().str.contains(query_title)]
        if matched_movies.empty:
            return "No movie found with that title."
        else:
            selected_movie = matched_movies.iloc[0]
            movie_index = selected_movie.name
            query_movie_embedding = movie_embeddings[movie_index]
            cosine_scores = util.cos_sim(query_movie_embedding, movie_embeddings)[0].cpu().numpy()
            cosine_scores[movie_index] = -np.inf
            top_indices = np.argsort(-cosine_scores)[:5]
            recommendations = "\n".join([f"{movie.iloc[idx]['title']} (Similarity Score: {cosine_scores[idx]:.4f})" for idx in top_indices])
            return recommendations
    elif query_type == "Genre/Description":
        user_query = input_text
        query_embedding = model.encode(user_query, convert_to_tensor=True)
        cosine_scores = util.cos_sim(query_embedding, movie_embeddings)
        cosine_scores = cosine_scores[0].cpu().numpy()
        top_indices = np.argsort(-cosine_scores)[:5]
        recommendations = "\n".join([f"{movie.iloc[idx]['title']} (Similarity Score: {cosine_scores[idx]:.4f})" for idx in top_indices])
        return recommendations
    else:
        return "Invalid query type."

iface = gr.Interface(
    fn=recommend_movies,
    inputs=[
        gr.Radio(["Movie Name", "Genre/Description"], label="Query Type"),
        gr.Textbox(label="Enter Movie Name or Genre/Description")
    ],
    outputs=gr.Textbox(label="Top 5 Recommendations"),
    title="Movie Recommendation Engine",
    description="Get movie recommendations based on movie name or genre/description."
)

iface.launch()
